# Installation

In [2]:
%%capture
!pip install tensorboard

# Importing the libraries

In [ ]:
import os
from collections import defaultdict
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import numpy as np

## Convert tesorboard to images
This function takes the directories for different experiments, and create images from the tensorboard files. The images are saved in the output directory given as an argument.

Inputs:
- `log_dirs`: List of directories containing the tensorboard files
- `output_dir`: Directory where the images will be saved

For more examples, check the example use case below

In [18]:
def tensorboard_to_images(log_dirs, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Dictionary to hold aggregated data for each metric
    aggregated_data = defaultdict(lambda: defaultdict(list))

    # Shortened names for legends
    legend_names = {
        'ForwardAndBackwardRaycastSoccerTwosRun': 'F&B Raycast',
        'OnlyForwardRaycastSoccerTwosRun': 'Only F Raycast',
        'SoundAndViewRotationSoccerTwosRun': 'Sound & View'
    }

    for log_dir in log_dirs:
        method_name = legend_names.get(os.path.basename(log_dir), os.path.basename(log_dir))
        for subdir, _, files in os.walk(log_dir):
            for file in files:
                if "events.out.tfevents" in file:
                    event_file = os.path.join(subdir, file)
                    ea = event_accumulator.EventAccumulator(event_file)
                    ea.Reload()
                    
                    for tag in ea.Tags()['scalars']:
                        events = ea.Scalars(tag)
                        steps = [e.step for e in events]
                        values = [e.value for e in events]
                        
                        # Append data to the corresponding metric (tag) and method
                        aggregated_data[tag][method_name].append((steps, values))
    
    # Create a separate plot for each metric (tag)
    for tag, methods in aggregated_data.items():
        plt.figure(figsize=(12, 8))  # Set high resolution for the output image
        
        # Plot all methods for this metric
        for method_name, data in methods.items():
            # Aggregate steps and values for each method
            all_steps = []
            all_values = []
            for steps, values in data:
                all_steps.extend(steps)
                all_values.extend(values)
            
            # Sort by steps and average values if needed
            sorted_indices = np.argsort(all_steps)
            sorted_steps = np.array(all_steps)[sorted_indices]
            sorted_values = np.array(all_values)[sorted_indices]
            
            # Optionally average values for repeated steps
            unique_steps, indices = np.unique(sorted_steps, return_inverse=True)
            averaged_values = np.bincount(indices, weights=sorted_values) / np.bincount(indices)

            # Plot the aggregated data
            plt.plot(unique_steps, averaged_values, label=f"{method_name}")
        
        plt.xlabel('Steps')
        plt.ylabel('Values')
        plt.title(f'TensorBoard Scalars - {tag}')
        plt.legend()
        
        # Save each plot to a separate file
        output_image_name = f"{tag.replace('/', '_')}.png"
        plt.savefig(os.path.join(output_dir, output_image_name), dpi=300)  # Save with high resolution
        plt.close()

In [ ]:
# Example usage
log_dirs = ["ForwardAndBackwardRaycastSoccerTwosRun", "OnlyForwardRaycastSoccerTwosRun", "SoundAndViewRotationSoccerTwosRun"]
output_dir = "results"
tensorboard_to_images(log_dirs, output_dir)